# Use Transfer learning with the multilingual BERT model to classify sentiment of Hebrew Texts

Source: https://github.com/shudima/notebooks/blob/master/hebrew_snetiment_analysis.ipynb?fbclid=IwAR1XRywUjrhTiHMCpzhxpflRcfawX5n1oA56LvI08TvLizswQK9XXoyvLGA

In [1]:


!pip install tamnun



     |████████████████████████████████| 13.9MB 237kB/s 
     |████████████████████████████████| 676.9MB 15kB/s 
     |████████████████████████████████| 5.4MB 13.8MB/s 
     |████████████████████████████████| 184kB 59.4MB/s 
     |████████████████████████████████| 1.1MB 41.0MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b80cd827ef0c263a35452ab6e7b1b4c19af04d154af31ce35bad554304171454
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.15.4 which is incompatible.
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirem

In [2]:
import tamnun

In [3]:
import codecs
import re
import numpy as np
from tamnun.bert import BertVectorizer, BertClassifier
import pandas as pd

In [4]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
from google.colab import files
import io
uploaded = files.upload()
temp = pd.read_csv(io.BytesIO(uploaded['sentences_list_shuffled_tagged_topics_2_train.csv']))
data = temp['text']
tags = temp['sentiment_tag'] + 1
data.shape, tags.shape


MessageError: ignored

In [23]:
def run_bert(train_tokens, train_tags, test_tokens, test_tags):
  vectorizer = BertVectorizer(do_truncate=True, bert_model='bert-base-multilingual-cased').fit(train_tokens)
  train_X = vectorizer.transform(train_tokens)
  clf = BertClassifier(num_of_classes=3, bert_model_name='bert-base-multilingual-cased', lr=1e-5).fit(train_X, train_tags)
  test_X = vectorizer.transform(test_tokens)
  predicted = clf.predict(test_X)
  accuracy = accuracy_score(test_tags, predicted)
  print(confusion_matrix(test_tags, predicted))
  return accuracy


In [24]:
uploaded = files.upload()
temp = pd.read_csv(io.BytesIO(uploaded['sentences_list_shuffled_tagged_topics_2_test.csv']))
data_test = temp['text']
run_bert(data, tags, data_test, None)

Saving sentences_list_shuffled_tagged_topics_2_test.csv to sentences_list_shuffled_tagged_topics_2_test (3).csv
Epoch 1/5:
77/78 batch loss: 0.6633488535881042 avg loss: 0.9997237569246537
Epoch 2/5:
77/78 batch loss: 0.472657710313797 avg loss: 0.9066177197755911
Epoch 3/5:
77/78 batch loss: 1.0919427871704102 avg loss: 0.7899028326456363
Epoch 4/5:
77/78 batch loss: 0.788998544216156 avg loss: 0.6716975884941908
Epoch 5/5:
77/78 batch loss: 0.18956363201141357 avg loss: 0.5522281174094249
['0, ', '1, ', '1, ', '0, ', '0, ', '1, ', '1, ', '0, ', '0, ', '1, ', '0, ', '0, ', '1, ', '1, ', '0, ', '0, ', '0, ', '0, ', '1, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '1, ', '0, ', '1, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '1, ', '1, ', '0, ', '0, ', '1, ', '0, ', '0, ', '0, ', '1, ', '0, ', '0, ', '0, ', '1, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '0, ', '1, ',

In [ ]:
TRAIN_SIZE = len(data)
n_folds = 10
def get_folds(X_train, y_train, num_folds=n_folds):
    fold_size = int(TRAIN_SIZE / num_folds)
    fold_idx = np.arange(fold_size, TRAIN_SIZE, fold_size)
    folds_X = []
    folds_y = []
    cur = 0
    for i in fold_idx:
        folds_X.append(X_train[cur:i])
        folds_y.append(y_train[cur:i])
        cur = i
    folds_X.append(X_train[cur:])
    folds_y.append(y_train[cur:])
    return np.array(folds_X), np.array(folds_y)

def CV(data, tags):
    folds_X, folds_y = get_folds(data, tags)
    mask = np.ones(len(folds_X), dtype=bool)
    accuracies = []
    for j in range(n_folds):
        mask[j] = False
        cur_X_train = np.concatenate(folds_X[mask])
        cur_y_train = np.concatenate(folds_y[mask])
        cur_X_test = folds_X[j]
        cur_y_test = folds_y[j]
        print(data.shape, cur_X_train.shape, cur_X_test.shape)
        mask[j] = True
        accuracy = run_bert(cur_X_train, cur_y_train, cur_X_test, cur_y_test)
        accuracies.append(accuracy)
    print('accuracies', accuracies)
    print('accuracy', np.average(accuracies))

CV(data, tags)

(314,) (283,) (31,)
Epoch 1/5:
70/71 batch loss: 0.8868464827537537 avg loss: 1.0261794941526063
Epoch 2/5:
70/71 batch loss: 0.9861459732055664 avg loss: 0.9750136674289972
Epoch 3/5:
70/71 batch loss: 1.3153938055038452 avg loss: 0.9006314781350149
Epoch 4/5:
70/71 batch loss: 0.9713189005851746 avg loss: 0.8391823944911151
Epoch 5/5:
70/71 batch loss: 0.09171418100595474 avg loss: 0.6633995340113908
[[ 2  2  1]
 [ 1  7  1]
 [ 0  7 10]]
(314,) (283,) (31,)
Epoch 1/5:
70/71 batch loss: 1.3881889581680298 avg loss: 1.0450740851147073
Epoch 2/5:
70/71 batch loss: 0.9396035671234131 avg loss: 0.9724878475699626
Epoch 3/5:
70/71 batch loss: 0.8754212260246277 avg loss: 0.8407854085237207
Epoch 4/5:
70/71 batch loss: 0.4300800561904907 avg loss: 0.6793962882857927
Epoch 5/5:
70/71 batch loss: 0.6534706354141235 avg loss: 0.5089660268014585
[[ 1  0  3]
 [ 3  4  6]
 [ 2  0 12]]
(314,) (283,) (31,)
Epoch 1/5:
70/71 batch loss: 1.105623722076416 avg loss: 1.0205140676296933
Epoch 2/5:
70/71 ba